## Conexão Spark com S3

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

AWS_ENDPOINT_URL = os.getenv('AWS_ENDPOINT_URL')
AWS_ACCESS_KEY = os.getenv('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.getenv('AWS_SECRET_KEY')

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf().setAppName("Spark com S3").setMaster("local[*]")

conf.set("spark.driver.memory", "100g")
conf.set("spark.executor.memory", "100g")
conf.set("spark.executor.pyspark.memory", "100g")

# conf.set("spark.driver.cores", "20")
# conf.set("spark.executor.cores", "20")

# conf.set("spark.memory.offHeap.enabled", "true")
# conf.set("spark.memory.offHeap.size", "20g")

# conf.set("spark.sql.shuffle.partitions", "2000")
# conf.set("spark.sql.parquet.columnarReaderBatchSize", "2048") 
conf.set("spark.sql.parquet.enableVectorizedReader", "false")
conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
conf.set("spark.sql.repl.eagerEval.enabled", "true")
conf.set("spark.sql.repl.eagerEval.truncate", 100)

conf.set("spark.hadoop.fs.s3a.access.key", AWS_ACCESS_KEY)
conf.set("spark.hadoop.fs.s3a.secret.key", AWS_SECRET_KEY)
conf.set("spark.hadoop.fs.s3a.endpoint", AWS_ENDPOINT_URL)
conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
conf.set("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.2.2")
conf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/home/darrazao/accounting_website_classifier/venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/darrazao/.ivy2/cache
The jars for the packages stored in: /home/darrazao/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4aba52e3-7b53-4fc7-a5b3-4fc44e5826bb;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 158ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	----------------------------

In [3]:
# df_spider_br = spark.read.parquet("s3a://drivalake/sites/bronze/spiderwebv4/brazil_*")
df_spider_countries = spark.read.parquet("s3a://drivalake/sites/bronze/spiderwebv4/countries_*", inferSchema=True)

24/05/14 17:54:28 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
from pyspark.sql.functions import col, size
df_spider_countries_br = df_spider_countries.select("domain", "html") 
df_spider_countries_br = df_spider_countries_br.withColumn("html", col("html").cast("string"))
df_spider_countries_br = df_spider_countries_br.filter((col("html") != '[]') & (col("html") != ''))
df_spider_countries_br = df_spider_countries_br.dropDuplicates()

In [5]:
df_spider_countries_br.printSchema()

root
 |-- domain: string (nullable = true)
 |-- html: string (nullable = true)



In [6]:
df_spider_countries_br
# 160min

24/05/14 18:09:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:09:47 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:11:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:11:13 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:13:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:13:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:14:43 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:15:07 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/05/14 18:15:07 WARN RowBasedKeyValueBatch: Calling spill() on

domain,html
0-0.repl.co,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n <title>Replit Legacy Hosting has migrated to Dep..."
00114.repl.co,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n <title>Replit Legacy Hosting has migrated to Dep..."
0013064.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
001294.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
010128.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
010403.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
002818.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
00-993.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
00-93s.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."
003719.carrd.co,"<!DOCTYPE html><html lang=""en-US""><head><title>Just a moment...</title><meta http-equiv=""Content-..."


In [8]:
# df_spider_br.count()
# 950.000
df_spider_countries_br.count()
# 

ERROR:root:KeyboardInterrupt while sending command.           (461 + 40) / 2992]
Traceback (most recent call last):
  File "/home/darrazao/accounting_website_classifier/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/darrazao/accounting_website_classifier/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_spider_countries_br.show()

ERROR:root:KeyboardInterrupt while sending command.            (98 + 40) / 2992]
Traceback (most recent call last):
  File "/home/darrazao/accounting_website_classifier/venv/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/darrazao/accounting_website_classifier/venv/lib/python3.10/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_spider_countries_br.count()
# 16.653.912 -> br, 7min
# 31.533.056 -> full

In [ ]:
# df_spider_br.filter(col('html') == '').count()
# 45 min, 514.889
# 4min, 514.889

514889

In [ ]:
# df_spider_br.head(3)

AttributeError: 'NoneType' object has no attribute 'setCallSite'

In [9]:
# Encerrar a sessão do Spark
spark.stop()

24/05/14 20:49:34 ERROR TaskSchedulerImpl: Exception in statusUpdate 40) / 2992]
java.util.concurrent.RejectedExecutionException: Task org.apache.spark.scheduler.TaskResultGetter$$anon$3@6077f06c rejected from java.util.concurrent.ThreadPoolExecutor@65ec7842[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 7088]
	at java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2063)
	at java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:830)
	at java.util.concurrent.ThreadPoolExecutor.execute(ThreadPoolExecutor.java:1379)
	at org.apache.spark.scheduler.TaskResultGetter.enqueueSuccessfulTask(TaskResultGetter.scala:61)
	at org.apache.spark.scheduler.TaskSchedulerImpl.liftedTree2$1(TaskSchedulerImpl.scala:836)
	at org.apache.spark.scheduler.TaskSchedulerImpl.statusUpdate(TaskSchedulerImpl.scala:811)
	at org.apache.spark.scheduler.local.LocalEndpoint$$anonfun$receive$1.applyOrElse(LocalSchedulerBackend.sc

24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/1f/temp_shuffle_0d0c5a57-84ee-4906-9551-de2e1d804205
24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/25/temp_shuffle_cbde955b-f386-4864-bf49-fcb769041258
24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/01/temp_shuffle_908f9019-785d-441e-adb0-0e68454874dd
24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/1d/temp_shuffle_8d65d01b-796a-436b-b504-4c2b0bcebf5a
24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/23/temp_shuffle_aacb531f-9423-42f9-89d7-fa6d7125292d
24/05/14 20:49:48 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-446bc885-4768-40d4-b0d5-65e6c668e65a/08/temp_shuffle_e6d8f656-3e26-4b9b-8669-79c117c260ac
24/05/14 2